In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

#!cp "/content/drive/My Drive/Dissertation/preprocessing.py" .
#!cp -r "/content/drive/My Drive/Dissertation/gym_maze" .
#!cp -r "/content/drive/My Drive/Dissertation/envs" .

Mounted at /content/drive/


In [ ]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/{name}" 

    torch.save({
      'meta_controller': model.meta_controller.state_dict(),
      'controller': model.controller.state_dict()
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/{name}" 
    checkpoint = torch.load(path)

    model.meta_controller.load_state_dict(checkpoint['meta_controller'])
    model.meta_controller_target = copy.deepcopy(model.meta_controller)
    model.controller.load_state_dict(checkpoint['controller'])
    model.controller_target = copy.deepcopy(model.controller)

    model.eval()
    model.meta_controller.eval()
    model.controller.eval()

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import cv2
from PIL import Image

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [ ]:
from cartpole import CartPoleEnv 
env = CartPoleEnv()
#env = gym.make("CartPole-v0")

---
### Helper functions

In [ ]:
def plot_durations():
    fig, axs = plt.subplots(2, figsize=(10,10))
    
    durations_t, durations = list(map(list, zip(*episode_durations)))
    durations = torch.tensor(durations, dtype=torch.float)
    visits_t = [(x[0], torch.tensor(x[1], dtype=torch.float)) for x in state_visits.items()]
    
    fig.suptitle('Training')
    axs[0].set_xlabel('Episode')
    axs[0].set_ylabel('Reward')
    axs[1].set_xlabel('Episode')
    axs[1].set_ylabel('State Visits')
    
    axs[0].plot(durations_t, durations.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 10:
        means = durations.unfold(0, 10, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(9), means))
        axs[0].plot(durations_t, means.numpy())
    
    if len(durations_t) >= 10:
        for t in visits_t:
            means = t[1].unfold(0, 10, 1).mean(1).view(-1)
            means = torch.cat((torch.zeros(9), means))
            axs[1].plot(means.numpy(), label=f"State {t[0]}")
            axs[1].legend()
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

---
### Code

In [ ]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
BATCH_SIZE = 16
GAMMA = 0.95

def one_hot(n, v):
    a = np.zeros(n)
    a[v] = 1.0
    return np.expand_dims(a, axis=0)

def rev_one_hot(a):
    return np.where(a[0] > 0)[0][0]

class DQN(nn.Module):
    def __init__(self, inputs, outputs, mem_len = 200000):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(inputs, 128)
        self.fc2 = nn.Linear(128, 128)
        self.head = nn.Linear(128, outputs)
        
        self.memory = ReplayMemory(mem_len)

        self.n_actions = outputs
        self.steps_done = 0
        
        self.EPS_START = 1.0
        self.EPS_END = 0.1
        self.EPS_DECAY = 1000 # in number of steps

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.head(x)
    
    def act(self, state, is_training=True):
        if is_training:
            eps_threshold = self.EPS_END + (self.EPS_START - self.EPS_END) * (1. - min(1., self.steps_done / self.EPS_DECAY))
            self.steps_done += 1

            # With probability eps select a random action
            if random.random() < eps_threshold:
                return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

        # otherwise select action = maxa Q∗(φ(st), a; θ)
        with torch.no_grad():
            return self(state).max(1)[1].view(1, 1)
    
    def experience_replay(self, optimizer, target):
        if len(self.memory) < BATCH_SIZE:
            return
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)
        
        current_Q_values = self(state_batch).gather(1, action_batch)
        # Compute next Q value based on which goal gives max Q values
        # Detach variable from the current graph since we don't want gradients for next Q to propagated
        next_max_q = target(next_state_batch).detach().max(1)[0]
        next_Q_values = not_done_mask * next_max_q
        # Compute the target of the current Q values
        target_Q_values = reward_batch + (GAMMA * next_Q_values)
        # Compute Bellman error (using Huber loss)
        loss = F.smooth_l1_loss(current_Q_values, target_Q_values.unsqueeze(1))
        
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        for param in self.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()
        
class HDQN(nn.Module):
    def __init__(self, inputs, outputs):
        super(HDQN, self).__init__()
        # Optimizer
        learning_rate = 2.5e-4
        
        # goal is left/right
        self.meta_controller = DQN(inputs, 2, mem_len = 100000).to(device)
        self.meta_controller_optimizer = optim.RMSprop(self.meta_controller.parameters(), lr=learning_rate)
        self.meta_controller_target = DQN(inputs, 2, mem_len = 0).to(device)
        self.meta_controller_target.eval()
        
        # takes goal+state jointly
        self.controller = DQN(inputs + 2, outputs, mem_len = 100000).to(device)
        self.controller_optimizer = optim.RMSprop(self.controller.parameters(), lr=learning_rate)
        self.controller_target = DQN(inputs + 2, outputs, mem_len = 0).to(device)
        self.controller_target.eval()
    
    def store_controller(self, *args):
        self.controller.memory.store(*args)
    
    def store_meta_controller(self, *args):
        self.meta_controller.memory.store(*args)
    
    def select_goal(self, external_observation, is_training=True):
        return self.meta_controller.act(external_observation, is_training)
        
    def select_action(self, joint_goal_obs, is_training=True):
        return self.controller.act(joint_goal_obs, is_training)
    
    def experience_replay(self):
        self.meta_controller.experience_replay(self.meta_controller_optimizer, self.meta_controller_target)
        self.controller.experience_replay(self.controller_optimizer, self.controller_target)

In [ ]:
def plot_norms(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    x, ys = np.array(list(episode_durations.keys())), np.array(list(episode_durations.values()))
    
    plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
    plt.xlabel('L2 Norm')
    plt.ylabel('Average Reward')
    
    mu = np.mean(ys, axis=1)
    plt.plot(x / 10, mu)
    stds = np.std(ys, axis = 1)
    plt.fill_between(x / 10, mu + stds , mu - stds, alpha=0.2)
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [ ]:
import time
SAVE_OFFSET = 1

def train_model():
    global SAVE_OFFSET
    # Get number of actions and observations from gym action space
    n_actions = env.action_space.n
    n_observations = env.observation_space.shape[0]

    # Initialize action-value function Q with random weights
    hdqnAgent = HDQN(n_observations, n_actions).to(device)

    max_episode_length = 500

    num_episodes = 2000 # M
    episode_durations = []
    state_visits = {0: [], 1: [], 2: [], 3: []}

    steps = 0
    for i_episode in range(num_episodes):
        observation = env.reset()

        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        overall_reward = 0
        ep_state_visits = {0: 0, 1: 0, 2: 0, 3: 0}
        done = False
        episode_steps = 0
        while not done:
            # select a goal
            goal = hdqnAgent.select_goal(state)
            goal_i = goal.item()
            encoded_goal = torch.from_numpy(one_hot(2, goal)).float().to(device)

            goal_done = False
            total_extrinsic = 0
            s_0 = state
            while not done and not goal_done:
                joint_goal_state = torch.cat([encoded_goal, state], axis=1)

                # Execute action a_t in emulator and observe reward r_t and image x_{t+1}
                action = hdqnAgent.select_action(joint_goal_state)
                action_i = action.item()

                observation, reward, done, _ = env.step(action_i)
                steps += 1
                extrinsic_reward = torch.tensor([reward], device=device)

                overall_reward += reward
                total_extrinsic += reward

                # preprocess φ_{t+1} = φ(s_{t+1})
                next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                joint_next_state = torch.cat([encoded_goal, next_state], axis=1)

                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                episode_steps += 1

                goal_done = (goal_i == action_i)

                if goal_done:
                    ep_state_visits[goal_i] += 1
                ep_state_visits[action_i + 2] += 1

                intrinsic_reward = torch.tensor([1.0 if goal_done else 0.0], device=device)

                # Store transition (φt, at, rt, φt+1) in D
                hdqnAgent.store_controller(joint_goal_state, action, joint_next_state, intrinsic_reward, done)

                state = next_state

                hdqnAgent.experience_replay()

            # Store transition for meta controller
            hdqnAgent.store_meta_controller(s_0, goal, next_state, torch.tensor([total_extrinsic], device=device), done)
            hdqnAgent.experience_replay()

        if i_episode % 20 == 0:
            hdqnAgent.meta_controller_target.load_state_dict(hdqnAgent.meta_controller.state_dict())
            hdqnAgent.controller_target.load_state_dict(hdqnAgent.controller.state_dict())
        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations)
        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if np.mean(dur[-100:]) >= 195:
                print(f"Solved after {i_episode} episodes!")
                save_model(hdqnAgent, f"hdqn_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return hdqnAgent

    return None # did not train

In [ ]:
agent = train_model()

Solved after 136 episodes!


In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data + eps * sign_data_grad * state_max

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), -state_max)

    return clipped_perturbed_data

def fgsm_goal(g_state, agent, eps, target, targetted):
    #g_state = torch.tensor(g_state, requires_grad=True)

    g_state_var = g_state.clone().detach().requires_grad_(True)

    # initial forward pass
    goal = agent.meta_controller(g_state_var)
    #goal = temp.max(1)[1].view(1, 1)

    if targetted:
        loss = F.smooth_l1_loss(goal, target)
    else:
        pass
        #loss = F.smooth_l1_loss(goal, temp.min(1)[1].view(1, 1).float())

    agent.meta_controller.zero_grad()

    # calc loss
    loss.backward()
    data_grad = g_state_var.grad.data

    # perturb state
    g_state_p = fgsm_attack(g_state, eps, data_grad)
    return agent.select_goal(g_state_p, False)

def fgsm_action(state, goal, agent, eps, target, targetted):
    #state = torch.tensor(state, requires_grad=True)
    state_var = state.clone().detach().requires_grad_(True)

    joint_goal_state = torch.cat([goal, state_var], 1).float()
    
    # initial forward pass
    action = agent.controller(joint_goal_state)
    #action = temp.max(1)[1].view(1, 1).float()

    if targetted:
        loss = F.smooth_l1_loss(action, target)
    else:
        pass
        #loss = F.smooth_l1_loss(action, temp.min(1)[1].view(1, 1).float())

    agent.controller.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state_var.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad)

    joint_goal_state = torch.cat([goal, state_p], 1).float()
    return agent.select_action(joint_goal_state, False)

def apply_fgsm(agent, episode_durations, goal_attack, action_attack, targetted):
    TARGET_GOAL = torch.tensor([[0.0, 0.0]], device=device, dtype=torch.float)
    TARGET_ACTION = torch.tensor([[0.0, 0.0]], device=device, dtype=torch.float)

    agent.eval()
    agent.meta_controller.eval()
    agent.controller.eval()

    max_episode_length = 200
    agent.meta_controller.is_training = False
    agent.controller.is_training = False

    num_episodes = 100

    for eps in np.arange(0.0, 0.031, 0.0025):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                
            episode_steps = 0
            done = False
            while not done:
                # select a goal
                if goal_attack:
                    goal = fgsm_goal(state, agent, eps, TARGET_GOAL, targetted)
                else:
                    goal = agent.select_goal(state, False)
                goal_i = goal.item()
                encoded_goal = torch.from_numpy(one_hot(2, goal)).float().to(device)

                goal_done = False
                while not done and not goal_done:
                    joint_goal_state = torch.cat([encoded_goal, state], axis=1)

                    if action_attack:
                        action = fgsm_action(state, encoded_goal, agent, eps, TARGET_ACTION, targetted)
                    else:
                        action = agent.select_action(joint_goal_state, False)

                    action_i = action.item()
                    observation, reward, done, _ = env.step(action_i)

                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    goal_done = (goal_i == action_i)

                    state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[eps].append(overall_reward / num_episodes)

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device)
def eval_model(hdqnAgent, episode_durations, goal_noise, action_noise, same_noise):
    hdqnAgent.eval()
    hdqnAgent.meta_controller.eval()
    hdqnAgent.controller.eval()

    max_episode_length = 200
    num_episodes = 100

    for l2norm in np.arange(0,0.31,0.03):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            noise = torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
            if goal_noise:
                g_state = state + state_max * noise
                g_state = g_state.float()
            if action_noise:
                if same_noise:
                    state = state + state_max * noise
                else:
                    state = state + state_max * torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
                state = state.float()

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal = hdqnAgent.select_goal(g_state, False)
                goal_i = goal.item()
                encoded_goal = torch.from_numpy(one_hot(2, goal)).float().to(device)

                goal_done = False
                while not done and not goal_done:
                    joint_goal_state = torch.cat([encoded_goal, state], axis=1)

                    action = hdqnAgent.select_action(joint_goal_state, False)
                    action_i = action.item()
                    observation, reward, done, _ = env.step(action_i)

                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    goal_done = (goal_i == action_i)

                    state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    noise = torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
                    if goal_noise:
                        g_state = state + state_max * noise
                        g_state = g_state.float()
                    if action_noise:
                        if same_noise:
                            state = state + state_max * noise
                        else:
                            state = state + state_max * torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
                        state = state.float()

        episode_durations[l2norm].append(overall_reward / num_episodes)

In [ ]:
def plot_fgsm(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    for kk in ['both', 'goal_only', 'action_only']:
        x, ys = np.array(list(episode_durations[kk].keys())), np.array(list(episode_durations[kk].values()))
        #plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
        plt.xlabel('$\epsilon$')
        plt.ylabel('Average Reward')
        
        mu = np.mean(ys, axis=1)
        plt.plot(x, mu, label=kk)
        stds = np.std(ys, axis = 1)
        plt.fill_between(x, mu + stds , mu - stds, alpha=0.2)
    
    plt.legend()
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [ ]:
targeted = {'both': {}, 'goal_only': {}, 'action_only': {}}
untargeted = {'both': {}, 'goal_only': {}, 'action_only': {}}
for eps in np.arange(0.0, 0.031, 0.0025):
    for x in ['both', 'goal_only', 'action_only']:
        targeted[x][eps] = []
        untargeted[x][eps] = []

n_actions = env.action_space.n
n_observations = env.observation_space.shape[0]

i = 5
while i < 25:
    #agent = train_model()
    agent = HDQN(n_observations, n_actions).to(device)
    load_model(agent, f"hdqn_{i}")
    if agent is not None:
        apply_fgsm(agent, targeted['both'], True, True, True)
        apply_fgsm(agent, targeted['goal_only'], True, False, True)
        apply_fgsm(agent, targeted['action_only'], False, True, True)
        #apply_fgsm(agent, untargeted['both'], True, True, False)
        #apply_fgsm(agent, untargeted['goal_only'], True, False, False)
        #apply_fgsm(agent, untargeted['action_only'], False, True, False)
        print(i)
        print(f"Targeted: {targeted}")
        print(f"Untargeted: {untargeted}")
        #plot_fgsm(episode_durations)
        i += 1

#plot_fgsm(episode_durations)
print(f"Targeted: {targeted}")
print(f"Untargeted: {untargeted}")

5
Targeted: {'both': {0.0: [200.0], 0.0025: [200.0], 0.005: [152.64], 0.0075: [144.88], 0.01: [169.75], 0.0125: [42.83], 0.015: [10.06], 0.0175: [9.57], 0.02: [9.52], 0.0225: [9.56], 0.025: [9.69], 0.0275: [9.55], 0.03: [9.71]}, 'goal_only': {0.0: [200.0], 0.0025: [200.0], 0.005: [200.0], 0.0075: [200.0], 0.01: [199.99], 0.0125: [194.8], 0.015: [29.12], 0.0175: [30.03], 0.02: [18.69], 0.0225: [26.55], 0.025: [23.72], 0.0275: [26.1], 0.03: [26.76]}, 'action_only': {0.0: [200.0], 0.0025: [200.0], 0.005: [153.54], 0.0075: [110.28], 0.01: [11.08], 0.0125: [11.27], 0.015: [10.18], 0.0175: [10.13], 0.02: [10.97], 0.0225: [10.2], 0.025: [10.35], 0.0275: [10.62], 0.03: [10.21]}}
Untargeted: {'both': {0.0: [], 0.0025: [], 0.005: [], 0.0075: [], 0.01: [], 0.0125: [], 0.015: [], 0.0175: [], 0.02: [], 0.0225: [], 0.025: [], 0.0275: [], 0.03: []}, 'goal_only': {0.0: [], 0.0025: [], 0.005: [], 0.0075: [], 0.01: [], 0.0125: [], 0.015: [], 0.0175: [], 0.02: [], 0.0225: [], 0.025: [], 0.0275: [], 0.03:

In [ ]:
same_noise = {}
diff_noise = {}
goal_only = {}
action_only = {}
for l2norm in np.arange(0,0.31,0.03):
    for i in [same_noise, diff_noise, goal_only, action_only]:
        i[l2norm] = []

# train 20 models, then eval them
i = 1
while i < 25:
    agent = train_model()
    if agent is not None:
        # goal_attack, action_attack, same_noise
        eval_model(agent, same_noise, True, True, True)
        eval_model(agent, diff_noise, True, True, False)
        eval_model(agent, goal_only, True, False, False)
        eval_model(agent, action_only, False, True, False)
        print(i)
        print(f"same noise: {same_noise}")
        print(f"diff noise: {diff_noise}")
        print(f"goal only: {goal_only}")
        print(f"action only: {action_only}")
        i += 1

print(f"same noise: {same_noise}")
print(f"diff noise: {diff_noise}")
print(f"goal only: {goal_only}")
print(f"action only: {action_only}")